# Overall Term Frequencies

With the TEDtalks-all dataset created, we have 1747 talks with which to work. This is a small corpus, and so the usual reasons for shrinking the feature set for the texts do not apply, but as we begin our survey of the contents of the TED talks we wanted to be mindful of standards that had emerged both so that our results would be comparable to the work of others but also so that we could potentially scale up the work here without having to re-think the foundations.

## Summary

In this notebook we load the complete dataset of both TED and TED+ talks. (No TEDx talks.) We use Python's `Sci-Kit Learn` library to create a document - term frequency matrix with a shape of 1747 x 50379. Summing the words to get a total for each word across all talks in the dataset, we then hand-inspect the totals and discover that there is a small list of numbers that recur. We approach mapping those numbers in two trials before returning to the task of getting a clean frequency list with no repetitions or other oddities.

## Imports and Data

In [1]:
# Imports
import pandas as pd, re, csv, nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Load the Data
df = pd.read_csv('../output/TEDall_speakers.csv')
df.shape

(1747, 27)

**To Do**: Edit the CSV to remove the vestigial index column at the start of each line. Then use `df.set_index('Talk_ID')`.

## Frequencies

The sole purpose of this notebook is to establish how we are going to elicit our features, our words, from the collection of talks. Thus, the only column we are interested in is the one with the texts of the talks. As we move forward, however, we will want to decide if we are simply going to append ~30,000 columns to a version of the extant CSV or create a separate CSV for each experiment. 

For this first experiment, we will keep it simple, creating two lists, one of the URLs and one of the texts. The URLs are unique, human-friendly identifiers for the talks. (We can, perhaps, make them a bit more friendly by modifying them a bit, subtracting `https://www.ted.com/talks/` from each.)

In [3]:
urls  = df.public_url.tolist()
texts = df.text.tolist()

There are a number of ways to get term frequencies, but **SciKit-Learn**'s `CountVectorizer` is, I think, the way to go, since it will work well with the other vectorizers and models also available in `sklearn`.

In our first experiment, we run `CountVectorizer` unadorned. The default options are: lowercase everything, get rid of all punctuation, make a word out of anything more than two characters long. The only thing that might not be welcome is the splitting of contractions. For now, we will leave things as they are. (Also, please note, no stopwords were used, so we have an unfiltered word list.)

For this current work, we are running `fit()` and `transform()` separately, but since `fit()` just calculates the parameters and saves them as an internal objects state `transform()`  applies the transformation to a particular set of examples (the ones we just fitted), the two operations are usually simply done at the same time as `fit_transform()`.

In [27]:
# If you want to pass options, pass them here:
vec = CountVectorizer()

# fit the model to the data 
vecs = vec1.fit(texts)

# transform the data according to the fitted model
X = vecs.transform(texts)

# see how many features we have
X.shape

(1747, 50379)

This is our baseline.

## Frequency Totals per Word

We can total up our columns for each feature (word), which is something we will be doing per year, per gender, per discipline. Here, we take the vector describing a word and sum it. We then pair the sum with the word in a tuple, which we then sort by frequency. 

(I'm doing it this way because it appears to be the way to do it, but it also strikes me that there should be a way to do this within the array itself, or, perhaps, to do it through **pandas**.)

We save the results to a CSV file so that we can hand-check the words: are these the results we expected? (We don't want any weirdness affecting our overall results.) The hand inspection looks good. I didn't see anything in words 4 or above in frequency that looked off. (So, the simplest solution works!) What I did note was the frequency of certain **numbers**: **100**, **12**, etc. This might be worth taking a closer look: are there *power numbers*? (I am thinking here of Alan Dundes' essay on the "power" of three in American culture.)

**To Do**: It would be nice to be able grab all words of a certain frequency, or range of frequencies.

---
**Follow-up**: whenever I attempt some version of
```python
for item in vecs:
    if vecs.vocabulary_.get(item) == 1691:
        print(item)
```
I get **`TypeError: 'CountVectorizer' object is not iterable`**. My best guess, for now, is that we need to use the tuple above to get this information.

---


In [ ]:
# summing up the counts for each word
sum_words = bow.sum(axis=0)

# create a tuple
words_freq = [(word, sum_words[0, idx]) for word, idx in vecs.vocabulary_.items()]

# sort the tuple
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

# check the results of our work by printing the top 20 more frequent words
print(words_freq[0:20])

In [ ]:
# with open('../output/word_freq.csv','w') as out:
#     csv_out = csv.writer(out)
#     csv_out.writerow(['word','count'])
#     for row in words_freq:
#         csv_out.writerow(row)

A hand inspection of the output above turned up a couple of interesting issues: that there are parentheticals mixed in with the text of the talks (see 01-Terms-02-Parentheticals) and numbers feature in the talks regularly (see 01-Terms-03-Numbers). 

There are also some interesting tokenization dimensions to a crowd-sourced transcription technology:

| Term    | Freq |
|:---------|---:|
|aa       |12 |
|aaa      |7  |
|aaaa     |2  |
|aaaaa    |1  |
|aaaaaaaah|1  |
|aaaaaaah |1  |
|aaaaaah  |3  |
|aaaah    |2  |
|aaaahhh  |1  |
|aaah     |3  |
|aag      |1  |
|aah      |10 |

There are an equal number of alternate spellings for *shh*. There's also this oddity:

| Term    | Freq |
|:---------|---:|
|FALSE    |115|
|TRUE     |909|

It appears to be simply the occurrences of *true* and *false* but somehow the vectorizer thinks they are Booleans?

In [ ]:
onetext = nltk.Text(re.sub("[^a-zA-Z0-9']"," ",'\n'.join(texts)).split())

In [ ]:
onetext.concordance("TRUE")

The rest of the hand-examination turns up no other issues, so the basic vectorization built into `sklearn` appears to be satisfactory, with the only exception being its breaking of contractions at the apostrophe: e.g., *isn't* becomes *isn* and *'t*, with the latter being thrown away as too small -- which means that the indefinite article *a* is also not present in the frequencies. 

## Revised Frequencies without Parentheticals

For more on the work leading up to this regex, see the `01-Terms-02-Parentheticals` notebook.

## Experiment 1

In [ ]:
# For more information on what these steps do, see the cell in Frequencies above.
vec = CountVectorizer(token_pattern=r'(?<=\().*?(?=\))')
X2 = vec.fit_transform(texts[235:237])
X2.shape

In [ ]:
pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [ ]:
# Here's what the texts look like unfiltered:
for text in texts[235:237]:
    print(text[0:80])

In [ ]:
# Here's the filter:
for text in texts[235:237]:
    retext = re.sub(r'\([^)]*\)', ' ', text)
    print(retext[0:80])

For now, without better regex-fu, we are going to filter the texts and then pass them onto the vectorizer:

In [ ]:
# This is a quick check of the list comprehension:
retexts = [ re.sub(r'\([^)]*\)', ' ', text) for text in texts[235:237]]

for text in retexts:
    print(text[0:80])

In [ ]:
# Everything above in one compact block:

# Filter texts to remove parentheticals
retexts = [ re.sub(r'\([^)]*\)', ' ', text) for text in texts ]
# Load and run the vectorizer
vec1 = CountVectorizer()
X1 = vec.fit_transform(retexts)
X1.shape

We have not dropped that many words: `50379 - 50316 = 63`, but we have removed words that were not spoken by the speakers. We now have a finished freqency count with which to begin the rest of our work. The cell below was used to write a new CSV -- it is commented out so that should anyone choose the "run all" option, then there won't be a danger of the files getting over-written. 

In [ ]:
# with open('../output/word_freq_noparens.csv','w') as out:
#     csv_out = csv.writer(out)
#     csv_out.writerow(['word','count'])
#     for row in words_freq:
#         csv_out.writerow(row)

**QUESTION**: Do we want to add these columns to the working CSV/dataframe? (Once we have a reasonable BoW representation of our texts, shouldn't we keep using the same one and not generate it every time we need it? Or, since we would only need certain columns, maybe a separate CSV?

## Experiment 2

Further experiments in the parentheticals notebook (01-Terms-02-Parentheticals), revealed that there is a a reasonable amount of speaker discourse being parenthesized, making removing all such parenthetical material less than optimal. However, the parentheticals appear to follow a Zipf distribution, and so we can effectively remove 80% of the parentheticals with a relatively small number of them fed to the vectorizer as stop words -- we will need to do so in a pre-processing step.

In [11]:
stopped_parens = [ "(laughter)", "(applause)", "(music)", "(video)", "(laughs)", 
                  "(applause ends)", "(audio)", "(singing)", "(music ends)", "(cheers)", 
                  "(cheering)", "(recording)", "(beatboxing)", "(audience)", 
                  "(guitar strum)", "(clicks metronome)", "(sighs)", "(guitar)", 
                  "(marimba sounds)", "(drum sounds)" ]
len(stopped_parens)

20

Success in this small-scale experiment here: `(laughter)` has been removed and `(whirring)` remains. We will scale this up now to the entirety of the corpus. 

First, we work out what code will give us the results we want and then, in the next cell, we use it with the vectorizer. (José Blanco has a post on _Towards Data Science_ on ["Hacking Scikit-Learn’s Vectorizers"](https://towardsdatascience.com/hacking-scikit-learns-vectorizers-9ef26a7170af).)

In [35]:
def paren_cleanse (text):
    result = ' '.join([ word for word in re.sub("[^a-zA-Z0-9'()]"," ", text).lower().split() 
              if word not in stopped_parens ])
    return result

clean_texts = [ paren_cleanse(text) for text in texts]
print(clean_texts[0][0:200])

thank you so much chris and it's truly a great honor to have the opportunity to come to this stage twice i'm extremely grateful i have been blown away by this conference and i want to thank all of you


'(mock', 'sob)'` is problematic. And, also a bit of a problem, `sklearn` expects input as strings -- I have not found a way to bypass this, save writing custom preprocessors or tokenizers, *à la*:

```python
def paren_cleanse(text):
    return(" ".join([ word for word in re.sub("[^a-zA-Z0-9'()]"," ", text.lower())
              if word not in stopped_parens ]))

vec3 = CountVectorizer(preprocessor = paren_cleanse)
```

A simpler approach might be to feed the vectorizer the stopped parenthetical tokens -- we do not know where the words are being removed in the process, and so one of the things will be to check the count of a word like *laughter* to see if its count decreased from one feature set to another.

In [37]:
vec2 = CountVectorizer(preprocessor = paren_cleanse)
X2 = vec2.fit_transform(texts)
X2.shape

(1747, 50301)

We are back to our original number of features. 

In [39]:
df_base = pd.DataFrame(X.toarray(), columns = vecs.get_feature_names())
df2 = pd.DataFrame(X2.toarray(), columns = vec2.get_feature_names())

In [41]:
laughter_1 = df_base['laughter'].sum()
laughter_2 = df2['laughter'].sum()
print(laughter_1, laughter_2)

7374 98


### Experiment 3

It would be really nice to be able to pass a "cleaned" text as a word list to sklearn's vectorizer, but it seems to accept only strings. 

In [15]:
# new_text = re.sub(stopped_parens, " ", texts[235])

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, ' ', new_text, flags=re.IGNORECASE)
    return new_text

print(clean_text(stopped_parens, texts[235]))

  Let's just get started here.    Okay, just a moment.    (Whirring)    All right. ( ) Oh, sorry.    ( ) ( )    Thank you.    ( )  


In [24]:
texts3 = [ clean_text(stopped_parens, text) for text in texts ]

In [25]:
vec3 = CountVectorizer()
X3 = vec3.fit_transform(texts3)
X3.shape

(1747, 50342)

In [29]:
df_base = pd.DataFrame(X.toarray(), columns = vecs.get_feature_names())
df3 = pd.DataFrame(X3.toarray(), columns = vec3.get_feature_names())

In [30]:
laughter_base = df_base['laughter'].sum()
laughter_3 = df3['laughter'].sum()
print(laughter_base, laughter_3)

KeyError: 'laughter'